In [ ]:
import os
import glob
import random

import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
import seaborn

In [ ]:
train_df = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
train_df

In [ ]:
print(f"There are {len(train_df)} patients in the dataset.")

In [ ]:
plt.figure(figsize=(5,5))
seaborn.countplot(data=train_df, x="MGMT_value")
print(train_df['MGMT_value'].value_counts())

In [ ]:
frq, bins, fig = plt.hist(train_df["BraTS21ID"])
plt.ylabel("frequency")
plt.xlabel("BraTSID")
plt.grid()
plt.show()
print("frequency array :", frq)
print("range array :", bins)

In [ ]:
root_dir = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/"
types = ["FLAIR","T1w","T1wCE","T2w"]
_new = []

for index, num in enumerate(train_df["BraTS21ID"]):
    a = []
    for i in range(len(types)):
        _path = root_dir + str(num).zfill(5)
        tpath = os.path.join(_path, types[i])
        lists = os.listdir(tpath)
        a.append(len(lists))
    
    _new.append([train_df["BraTS21ID"].iloc[index],train_df["MGMT_value"].iloc[index],a[0], a[1], a[2], a[3]])

new_df = pd.DataFrame(_new)
new_df.columns = ["BraTS21Id", "MGMT_value","FLAIR", "T1w", "T1wCE", "T2w"]

new_df

In [ ]:
def dicom2array(path, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    
    # depending on value, image can be look inverted, so fix this
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    
    data = data - np.min(data)
    data = data / np.max(data)    # 0~1
    data = (data * 255).astype(np.uint8)    # 0~255
    
    return data

In [ ]:
def visualize_sample(
    brats21id, 
    slice_i, 
    mgmt_value, 
    types=("FLAIR", "T1w", "T1wCE", "T2w")
):
    plt.figure(figsize=(16, 5))
    
    patient_path = os.path.join(
        "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/", 
        str(brats21id).zfill(5)
    )
    
    for i, t in enumerate(types, 1):
        t_paths = sorted(
            glob.glob(os.path.join(patient_path, t, "*")),
            key=lambda x: int(x[:-4].split("-")[-1])
        )
        data = dicom2array(t_paths[int(len(t_paths) * slice_i)])
        plt.subplot(1, 4, i)
        plt.imshow(data, cmap="gray")
        plt.title(f"{t}", fontsize=16)
        plt.axis("off")
        
    plt.suptitle(f"MGMT_value: {mgmt_value}", fontsize=16)
    plt.show()

In [ ]:
for i in random.sample(range(train_df.shape[0]), 5):
    _brats21id = train_df.iloc[i]["BraTS21ID"]
    _mgmt_value = train_df.iloc[i]["MGMT_value"]
    visualize_sample(brats21id=_brats21id, mgmt_value=_mgmt_value, slice_i=0.5)

In [ ]:
for i in random.sample(range(train_df.shape[0]), 5):
    _brats21id = train_df.iloc[i]["BraTS21ID"]
    _mgmt_value = train_df.iloc[i]["MGMT_value"]
    visualize_sample(brats21id=_brats21id, mgmt_value=_mgmt_value, slice_i=0.3)

In [ ]:
from matplotlib import animation, rc
rc('animation', html='jshtml')


def create_ani(ims):
    fig = plt.figure(figsize=(6,6))
    plt.axis('off')
    im = plt.imshow(ims[0], cmap="gray")
    
    def animate_func(i):
        im.set_array(ims[i])
        return [im]
    
    return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=1000//24)

In [ ]:
def load_dicom_line(path):
    t_paths = sorted(
            glob.glob(os.path.join(path, "*")),
            key=lambda x: int(x[:-4].split("-")[-1])
        )
    
    images = []
    for filename in t_paths:
        data = dicom2array(filename)
        if data.max() == 0:
            continue
        images.append(data)
    
    return images

In [ ]:
images = load_dicom_line("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00002/FLAIR")
create_ani(images)